In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline
from joblib import Memory
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [3]:
pip install prefect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from prefect import task, flow

In [18]:
from sklearn import metrics

In [19]:
@task
def load_data(file_path):
    """
    Load data from a CSV file.
    """
    return pd.read_csv(file_path)


@task
def split_inputs_output(data, inputs, output):
    """
    Split features and target variables.
    """
    X = data[inputs]
    y = data[output]
    return X, y


@task
def split_train_test(X, y, test_size=0.25, random_state=0):
    """
    Split data into train and test sets.
    """
    return train_test_split(X, y, test_size=test_size, random_state=random_state)


@task
def preprocess_data(X_train, X_test, y_train, y_test):
    """
    Rescale the data.
    """
    vocab = CountVectorizer()
    X_train_num = vocab.fit_transform(X_train)
    X_test_num = vocab.transform(X_test)
    return X_train_num, X_test_num, y_train, y_test


@task
def train_model(X_train_num, y_train, hyperparameters):
    """
    Training the machine learning model.
    """
    clf = MultinomialNB()
    clf.fit(X_train_num, y_train)
    return clf


@task
def evaluate_model(model, X_train_num, y_train, X_test_num, y_test):
    """
    Evaluating the model.
    """
    y_train_pred = model.predict(X_train_num)
    y_test_pred = model.predict(X_test_num)

    train_score = metrics.accuracy_score(y_train, y_train_pred)
    test_score = metrics.accuracy_score(y_test, y_test_pred)

    return train_score, test_score

In [20]:
# Workflow

@flow(name="Naivebayes Training Flow")
def workflow():
    DATA_PATH = "/content/my_data.csv"
    INPUTS = 'Review text'
    OUTPUT = 'Sentiment'
    HYPERPARAMETERS = {'vectorization': [CountVectorizer()]}

    # Load data
    iris = load_data(DATA_PATH)

    # Identify Inputs and Output
    X, y = split_inputs_output(iris, INPUTS, OUTPUT)

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = split_train_test(X, y)

    # Preprocess the data
    X_train_num, X_test_num, y_train, y_test = preprocess_data(X_train, X_test, y_train, y_test)

    # Build a model
    model = train_model(X_train_num, y_train, HYPERPARAMETERS)

    # Evaluation
    train_score, test_score = evaluate_model(model, X_train_num, y_train, X_test_num, y_test)

    print("Train Score:", train_score)
    print("Test Score:", test_score)

In [21]:
if __name__ == "__main__":
    workflow()

10:49:54.131 | INFO    | prefect.engine - Created flow run 'talented-swine' for flow 'Naivebayes Training Flow'

10:49:54.301 | INFO    | Flow run 'talented-swine' - Created task run 'load_data-0' for task 'load_data'

10:49:54.305 | INFO    | Flow run 'talented-swine' - Executing 'load_data-0' immediately...

10:49:54.512 | INFO    | Task run 'load_data-0' - Finished in state Completed()

10:49:54.575 | INFO    | Flow run 'talented-swine' - Created task run 'split_inputs_output-0' for task 'split_inputs_output'

...

10:49:55.853 | INFO    | Flow run 'talented-swine' - Created task run 'evaluate_model-0' for task 'evaluate_model'

10:49:55.858 | INFO    | Flow run 'talented-swine' - Executing 'evaluate_model-0' immediately...

10:49:56.030 | INFO    | Task run 'evaluate_model-0' - Finished in state Completed()

Train Score: 0.9505652100556774
Test Score: 0.930161943319838


10:49:56.114 | INFO    | Flow run 'talented-swine' - Finished in state Completed('All states completed.')